In [43]:
# Imports
from whoosh.fields import *
from whoosh.index import create_in, open_dir
from whoosh.qparser import MultifieldParser, OrGroup, AndGroup
from whoosh.query import *

import sys, os, threading, time, codecs
import numpy as np
import pandas as pd

from datetime import datetime
import uuid
from preprocessing import *
import abc
import copy
import csv
import json
import os.path
import sys
import utils

In [52]:
# Open csv file
col = ['timestamp', 'field_1', 'field_2', 'field_3', 'field_4',
      'field_5', 'field_6', 'field_7', 'field_8', 'field_9']
file = pd.read_csv('JH3iid1bZ1Q.classify.000-600.csv', names = col)

# Create a schema
schema = Schema(
    timestamp = TEXT (stored = True,  field_boost = 2.0),
    field_1 = TEXT (stored = True,  field_boost = 2.0),
    field_2 = TEXT (stored = True,  field_boost = 2.0),
    field_3 = TEXT (stored = True,  field_boost = 2.0),
    field_4 = TEXT (stored = True,  field_boost = 2.0),
)

# Check if index is created
if not os.path.exists("Video_Index"):
    os.mkdir('Video_Index')
ix = create_in("Video_Index", schema)

In [53]:
# Populate the Index
with ix.writer() as writer:
    for i in file.index:
        writer.update_document(
            timestamp = str(file.loc[i, 'timestamp']),
            field_1 = str(file.loc[i, 'field_1']), 
            field_2 = str(file.loc[i, 'field_2']),
            field_3 = str(file.loc[i, 'field_3']),
            field_4 = str(file.loc[i, 'field_4']))

In [62]:
# Search the index
# Create and Or factor parser
og = AndGroup
mp = MultifieldParser(
                    ["timestamp",  'field_1', 'field_2', 'field_3', 'field_4'],
                    ix.schema,
                    group = og)

# Define the query and search
q = mp.parse("Swimming Beach")
with ix.searcher() as s:
    results = s.search(q, limit = 1)
    print("Search Results: ")
    for i in range(1):
        print(results[i])

Search Results: 
<Hit {'field_1': 'Human swimming:0.0547697', 'field_2': 'Resort:0.0207724', 'field_3': 'Beach:0.0194464', 'field_4': 'Swimming pool:0.0183581', 'timestamp': '184000000'}>


In [66]:
print("Search Results: ")
print("Swimming Beach: ", "video:", "JH3iid1bZ1Q", "start: 144000000")

Search Results: 
Swimming Beach:  video: JH3iid1bZ1Q start: 144000000


In [1]:
!pip install elasticsearch

     |████████████████████████████████| 92kB 6.9MB/s eta 0:00:011


In [20]:
!python csv_to_elastic.py \
    --elastic-address 'localhost:9200' \
    --csv-file /work/ActIQ/product/engine/cache/ydiALQ7XHd0.combined.000-350.csv \
    --elastic-index 'actiq' \
    --elastic-type "video" \
    --datetime-field=dateField \
    --delimiter ',' \
    --id-column id \
    --json-struct '{"ind" : "%name%", "video" : "%video%", "stamp" : "%stamp%", "model" : "%model%", "rank"  : "%rank%","prob"  : "%prob%", "text"  : "%text%"}'


 ----- CSV to ElasticSearch ----- 
Importing all rows into `index` from '/work/ActIQ/product/engine/cache/_XmhBaUdges.combined.000-060.csv'

Reached end of CSV - sending to Elastic
Using unencrypted http
Traceback (most recent call last):
  File "/work/ActIQ/product/engine/app/csv_to_elastic.py", line 233, in <module>
    elastic_address=parsed_args.elastic_address, ssl=parsed_args.ssl, username=parsed_args.username, password=parsed_args.password, id_column=parsed_args.id_column)
  File "/work/ActIQ/product/engine/app/csv_to_elastic.py", line 129, in main
    send_to_elastic(elastic_address, endpoint, ssl, username, password, to_elastic_string, count)
  File "/work/ActIQ/product/engine/app/csv_to_elastic.py", line 151, in send_to_elastic
    connection.request('POST', url=endpoint, headers = headers, body=to_elastic_string)
  File "/opt/conda/envs/rapids/lib/python3.7/http/client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/opt

In [38]:
import unidecode
accented_string = u'Málaga'
unaccented_string = unidecode.unidecode(accented_string)

ModuleNotFoundError: No module named 'unidecode'

In [2]:
from elasticsearch import Elasticsearch 
    es=Elasticsearch([{'host':'localhost','port':9200}])

In [3]:
es.indices.get_alias("*")

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7fc84ec8e518>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7fc84ec8e518>: Failed to establish a new connection: [Errno 111] Connection refused)

In [9]:
import csv
import sys
import os
import re
import json
import requests
from elasticsearch import Elasticsearch

dataFilePath = '/work/ActIQ/product/engine/cache/'

class ElasticSearchImporter(object):
    def importToDb(self, fileName, indexDbName, indexType="default"):
        csv.field_size_limit(sys.maxsize)
        es = Elasticsearch()
        list = []
        headers = []
        index = 0

        f = open(dataFilePath+fileName, 'rb')
        reader = csv.reader(f)

        try:
            for row in reader:
                try:
                    if(index == 0):
                        headers = row
                    else:
                        obj = {}
                        for i, val in enumerate(row):
                            if header == "rank":
                                row[i] = str(int(row[i])+1)
                            if header == "stamp":
                                row[i] = str(int(row[i]))
                            obj[headers[i]] = val
                        es.index(index=indexDbName, doc_type=indexType, body=obj)

                except Exception as e:
                    print(index)
                    print(e)

                index = index + 1
        
        except:
            print ('error')

        if not f.closed:
            f.close()

importer = ElasticSearchImporter()
importer.importToDb("_XmhBaUdges.combined.000-060.csv", "testIndex", indexType="default")

error


In [10]:
import argparse
import http.client
import os
import csv
import json
import dateutil.parser
from base64 import b64encode


def main(file_path, delimiter, max_rows, elastic_index, json_struct, datetime_field, elastic_type, elastic_address, ssl, username, password, id_column):
    endpoint = '/_bulk'
    if max_rows is None:
      max_rows_disp = "all"
    else:
      max_rows_disp = max_rows

    print("")
    print(" ----- CSV to ElasticSearch ----- ")
    print("Importing %s rows into `%s` from '%s'" % (max_rows_disp, elastic_index, file_path))
    print("")

    count = 0
    headers = []
    headers_position = {}
    to_elastic_string = ""
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=delimiter, quotechar='"')
        for row in reader:
            if count == 0:
                for iterator, col in enumerate(row):
                    headers.append(col)
                    headers_position[col] = iterator
            elif max_rows is not None and count >= max_rows:
                print('Max rows imported - exit')
                break
            elif len(row[0]) == 0:    # Empty rows on the end of document
                print("Found empty rows at the end of document")
                break
            else:
                pos = 0
                if os.name == 'nt':
                    _data = json_struct.replace("^", '"')
                else:
                    _data = json_struct.replace("'", '"')
                _data = _data.replace('\n','').replace('\r','')
                for header in headers:
                    if header == datetime_field:
                        datetime_type = dateutil.parser.parse(row[pos])
                        _data = _data.replace('"%' + header + '%"', '"{:%Y-%m-%dT%H:%M}"'.format(datetime_type))
                    if header == "rank":
                        row[pos] = str(int(row[pos])+1)
                        _data = _data.replace('"%' + header + '%"', row[pos])
                    if header == "stamp":
                         _data = _data.replace('"%' + header + '%"', str(int(row[pos])))
                    else:
                        try:
                            int(row[pos])
                            _data = _data.replace('"%' + header + '%"', row[pos])
                        except ValueError:
                            _data = _data.replace('%' + header + '%', row[pos])
                    pos += 1
                # Send the request
                if id_column is not None:
                    index_row = {"index": {"_index": elastic_index,
                                           "_type": elastic_type,
                                           '_id': row[headers_position[id_column]]}}
                else:
                    index_row = {"index": {"_index": elastic_index, "_type": elastic_type}}
                json_string = json.dumps(index_row) + "\n" + _data + "\n"
                to_elastic_string += json_string
            count += 1
            if count % 10000 == 0:
                send_to_elastic(elastic_address, endpoint, ssl, username, password, to_elastic_string, count)
                to_elastic_string = ""

    print('Reached end of CSV - sending to Elastic')
    send_to_elastic(elastic_address, endpoint, ssl, username, password, to_elastic_string, count)

    print("Done.")

def send_to_elastic(elastic_address, endpoint, ssl, username, password, to_elastic_string, block=0):

    if ssl:
        print("Using HTTPS")
        connection = http.client.HTTPSConnection(elastic_address)
    else:
        print("Using unencrypted http")
        connection = http.client.HTTPConnection(elastic_address)

    headers = {"Content-type": "application/json", "Accept": "text/plain"}

    if not username is None:
        auth = b64encode(bytes("{}:{}".format(username,password), "utf-8")).decode("ascii")
        if ssl or "localhost" in elastic_address:
            headers['Authorization'] = "Basic {}".format(auth)
        else:
            print("*** Warning: Refusing to remotely transmit user/pass unencrypted.")

    connection.request('POST', url=endpoint, headers = headers, body=to_elastic_string)
    response = connection.getresponse()
    body = response.read().decode('utf-8')
    response_details=json.loads(body)

    if response.status != 200:
        print ("\n*** Error occured before import. ***")
        print ("HTTP error code {} / {}".format(response.status, response.reason))

    else:

        if response_details['errors']:
            line=1 # skip header 
            for i in response_details['items']:
                line=line+1
                if i['index']['status'] != 201:
                    print("\n*** Problem on line {}: {}".format(block+line,i['index']['error']))
            print ("\n*** Error occured during import. See response body above. ***\n")
        else:
            print ("Import of {} items was successful.".format(len(response_details['items'])))
        return


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='CSV to ElasticSearch.')

    parser.add_argument('--elastic-address',
                        required=False,
                        type=str,
                        default='localhost:9200',
                        help='Your elasticsearch endpoint address')
    parser.add_argument('--ssl',
                        dest='ssl',
                        action='store_true',
                        required=False,
                        help='Use SSL connection')
    parser.add_argument('--username',
                        required=False,
                        type=str,
                        help='Username for basic auth (for example with elastic cloud)')
    parser.add_argument('--password',
                        required=False,
                        type=str,
                        help='Password')
    parser.add_argument('--csv-file',
                        required=True,
                        type=str,
                        help='path to csv to import')
    parser.add_argument('--json-struct',
                        required=True,
                        type=str,
                        help='json to be inserted')
    parser.add_argument('--elastic-index',
                        required=True,
                        type=str,
                        help='elastic index you want to put data in')
    parser.add_argument('--elastic-type',
                        required=False,
                        type=str,
                        default='test_type',
                        help='Your entry type for elastic')
    parser.add_argument('--max-rows',
                        type=int,
                        default=None,
                        help='max rows to import')
    parser.add_argument('--datetime-field',
                        type=str,
                        help='datetime field for elastic')
    parser.add_argument('--id-column',
                        type=str,
                        default=None,
                        help='If you want to have index and you have it in csv, this the argument to point to it')
    parser.add_argument('--delimiter',
                        type=str,
                        default=";",
                        help='If you want to have a different delimiter than ;')

    parsed_args = parser.parse_args()

    main(file_path=parsed_args.csv_file, delimiter = parsed_args.delimiter, json_struct=parsed_args.json_struct,
         elastic_index=parsed_args.elastic_index, elastic_type=parsed_args.elastic_type,
         datetime_field=parsed_args.datetime_field, max_rows=parsed_args.max_rows,
         elastic_address=parsed_args.elastic_address, ssl=parsed_args.ssl, username=parsed_args.username, password=parsed_args.password, id_column=parsed_args.id_column)

usage: ipykernel_launcher.py [-h] [--elastic-address ELASTIC_ADDRESS] [--ssl]
                             [--username USERNAME] [--password PASSWORD]
                             --csv-file CSV_FILE --json-struct JSON_STRUCT
                             --elastic-index ELASTIC_INDEX
                             [--elastic-type ELASTIC_TYPE]
                             [--max-rows MAX_ROWS]
                             [--datetime-field DATETIME_FIELD]
                             [--id-column ID_COLUMN] [--delimiter DELIMITER]
ipykernel_launcher.py: error: the following arguments are required: --csv-file, --json-struct, --elastic-index


SystemExit: 2

/opt/conda/envs/rapids/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3299: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [42]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

search = es.search(index="actiq", body={"query": {"match" : {"text": "rookie quarterback"}}})

print("Got %d Hits:" % search['hits']['total']['value'])
for hit in search['hits']['hits']:
    print("%(video)s %(stamp)s: %(prob)s" % hit["_source"])

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7fc84eac3e80>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7fc84eac3e80>: Failed to establish a new connection: [Errno 111] Connection refused)

In [ ]:
curl -XDELETE localhost:9200/actiq
curl -GET localhost:9200/actiq/_search?q=text:rookie
curl -GET localhost:9200/actiq/_count